In [ ]:
import numpy as np
import dill
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.collections import LineCollection
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
import auxiliaries as aux

In [ ]:
class EmptyAgent:
    def __init__(self):
        pass

def load_data(path):
    import dill
    with open(path, 'rb') as f:
        data = dill.load(f)
        
    print(f"Data loaded from {path}.")
    print(f"Number of agents: {data['MAS_parameters']['num_agents']}")
    print(f"MAS type: {data['MAS_parameters']['MAS_type']}")
    
    agents = []
    for agentid in data['agents']:
        agent = EmptyAgent()
        agent.id = agentid
        agent.cl_x = data['agents'][agentid]['cl_x']
        agent.cl_u = data['agents'][agentid]['cl_u']
        if 'MPC_sol' in data['agents'][agentid]:
            agent.MPC_sol = data['agents'][agentid]['MPC_sol']
        agents.append(agent)
        
    return data, agents

def extract_keys(d, parent_key=""):
    keys = set()
    
    if isinstance(d, dict):
        for key, value in d.items():
            full_key = f"{parent_key}.{key}" if parent_key else key
            keys.add(full_key)
            keys.update(extract_keys(value, full_key))
    
    return keys

In [ ]:
"""Select colours for plotting."""
colours = [
    "#0072B2",  # blue
    "#D55E00",  # orange
    "#009E73",  # green
    "#CC79A7",  # magenta
    "#56B4E9",  # light blue
    "#E69F00",  # yellow-orange
    "#B22222",  # red
    "#6A3D9A",  # purple
    "#117733",  # teal green
    "#88CCEE",  # cyan
    "#DDCC77",  # muted yellow-orange
]

In [ ]:
path = "./data/quadrotor_data.dill"   # Specify the path to your data file.    
animate = False                       # Set to True if you want to generate an animation (requires ffmpeg).

data, agents = load_data(path)
print("Discretization:", data['MAS_parameters']['h'])
for key in data['sim_pars']:
    print(f"{key}:", data['sim_pars'][key])

In [ ]:
"""Extract and transform data."""
max_sim_time = data['sim_data']['max_sim_time']

if type(data['sim_data']['cooperative_cost']) is list:
    data['sim_data']['cooperative_cost'] = np.vstack(data['sim_data']['cooperative_cost']).flatten()
    data['sim_data']['tracking_cost'] = np.vstack(data['sim_data']['tracking_cost']).flatten()
    data['sim_data']['change_cost'] = np.vstack(data['sim_data']['change_cost']).flatten()
    data['sim_data']['J'] = np.vstack(data['sim_data']['J']).flatten()
    
for agent in agents:
    if type(agent.cl_x) == list:
        agent.cl_x = np.hstack(agent.cl_x)
        agent.cl_u = np.hstack(agent.cl_u)
    

In [ ]:
"""Plot the value function."""
# Plot from t1 to t2.
t1 = 0
t2 = max_sim_time+1

# Select a feasible start time (the end time is controlled below).
t1 = min(t1, max_sim_time+1)

# Draw the evolution in state space:
fig_V, ax_V = plt.subplots(figsize=(10, 6), num='state evolution')

stop_time = data['sim_data']['cooperative_cost'][t1:t2].shape[0]
ax_V.plot(range(t1, min(t2, stop_time)), data['sim_data']['cooperative_cost'][t1:t2], label='cooperative', color=colours[0])
ax_V.plot(range(t1, min(t2, stop_time)), data['sim_data']['tracking_cost'][t1:t2], label='tracking', color=colours[1])
ax_V.plot(range(max(t1,1), min(t2, stop_time)), data['sim_data']['change_cost'][max(t1,1):t2], label='change', color=colours[2])
ax_V.plot(range(t1, min(t2, stop_time)), data['sim_data']['J'][t1:t2], '--', label='J', color=colours[3])
    
ax_V.set_xlabel('time steps')
ax_V.set_title(f'Value function over time')
ax_V.grid(True)
ax_V.legend()

# Set the y-axis to logarithmic scale.
ax_V.set_yscale('log')

plt.show()

print(f'Value function difference between the first and last time step: {data["sim_data"]["J"][-1] - data["sim_data"]["J"][0]}')
print(f'Value function at start: {data["sim_data"]["J"][0]:15.4e}')
print(f'Value function at stop : {data["sim_data"]["J"][-1]:15.4e}; diff: {data["sim_data"]["J"][-1] - data["sim_data"]["J"][0]:15.4e}')
print(f'Cooperation cost at start : {data["sim_data"]["cooperative_cost"][0]:15.4e}')
print(f'Cooperation cost at stop : {data["sim_data"]["cooperative_cost"][-1]:15.4e}; diff: {data["sim_data"]["cooperative_cost"][-1] - data["sim_data"]["cooperative_cost"][0]:15.4e}')


In [ ]:
"""2D position"""
# Plot from t1 to t2.
t1 = 0
t2 = agent.cl_x.shape[1]-1
step = 1

# Select a feasible start time (the end time is controlled automatically).
t1 = min(t1, max_sim_time+1)

# Draw the evolution in state space:
fig_cl, ax_cl = plt.subplots(figsize=(7, 7), num='state evolution')

for i, agent in enumerate(agents):
    cl_x = np.zeros(agent.cl_x.shape)
    # For the constellation task, transform the polar coordinates into Cartesian coordinates.
    cl_x = agent.cl_x
    ax_cl.plot(cl_x[0, t1 : t2+1:step], cl_x[1,t1 : t2+1:step], color=colours[i], label=f'{agent.id}_x', 
               #marker='o', markersize=2, 
               linewidth=1.5)
    # Mark the initial state with a larger circle.
    ax_cl.plot(cl_x[0,t1], cl_x[1,t1], color=colours[i], marker='o', markersize=6)
    # Mark the final state with a cross.
    ax_cl.plot(cl_x[0,t2], cl_x[1,t2], color=colours[i], marker='x', markersize=6)

    if f'{agent.id}_radius' in agent.MPC_sol:
        radius = agent.MPC_sol[f'{agent.id}_radius']
        yT_centre = agent.MPC_sol[f'{agent.id}_yT_centre']
        # Sample the circle.
        circle_phi = np.linspace(0, 2 * np.pi, 100)
        circle_x = radius * np.cos(circle_phi) + yT_centre[0]
        circle_y = radius * np.sin(circle_phi) + yT_centre[1]
        ax_cl.plot(circle_x.T, circle_y.T, color=colours[i], linestyle=':')
    
    # Plot the final cooperation output.
    ax_cl.plot(data['sim_data']['yT'][f'{agent.id}'][-1][0, :], data['sim_data']['yT'][f'{agent.id}'][-1][1, :], color=colours[i], markersize=2, linewidth=1, marker='o', label=f'A{agent.id}_yT', alpha=0.25)
    
# Plot the last cooperation output.
# for i, agent in enumerate(agents):
#     yT_cl = data['sim_data']['yT'][f'{agent.id}'][-1]
#     ax_cl.plot(yT_cl[0,:], yT_cl[1,:], color=colours[i], markersize=3, linewidth=0.5, marker='o', label=f'A{agent.id}_yT')


ax_cl.set_xlabel('$x_1$')    
ax_cl.set_ylabel('$x_2$')
ax_cl.set_title(f'Closed-loop position from $t = {t1}$ to $t = {t2}$ with step {step}')
ax_cl.set_xlim([-2.5, 2.5])
ax_cl.set_ylim([-2.5, 2.5])
ax_cl.grid()

plt.show()

In [ ]:
"""All states and inputs"""

# Define time range
t1 = 0
t2_state = max_sim_time + 1
t2_input = max_sim_time

# Number of states and inputs
num_states = 10
num_inputs = 3

# Select feasible start time
t1 = min(t1, max_sim_time + 1)

# Extract the time of the switch.
t_switch = data['cooperative_task']['switching_time']

# Create a figure with multiple subplots
fig, axes = plt.subplots(7, 2, figsize=(12, 16), num='State & Input Evolution')

## --- Plot All 6 States ---
for idx_state in range(num_states):
    ax = axes[idx_state // 2, idx_state % 2]  # Get subplot position
    title_state = f'Closed-loop state $x_{idx_state+1}$ from t = {t1} to t = {t2_state}'

    for i, agent in enumerate(agents):
        tf = min(t2_state, agent.cl_x.shape[1] - 1)
        if idx_state == 3 or idx_state == 4:
            ax.plot(range(t1, tf+1), np.degrees(agent.cl_x[idx_state, t1:tf+1]), 
                    color=colours[i], label=f'{agent.id}_x{idx_state+1}', markersize=0, linewidth=2, marker='o')
        else:
            ax.plot(range(t1, tf+1), agent.cl_x[idx_state, t1:tf+1], 
                    color=colours[i], label=f'{agent.id}_x{idx_state+1}', markersize=0, linewidth=2, marker='o')

    if np.linalg.norm(ax.get_ylim()) < 1e-8:
        ax.set_ylim(-0.1, 0.1)

    ax.set_xlabel('$t$')
    ax.set_ylabel(f'$x_{idx_state+1}$')
    ax.set_title(title_state)
    # ax.legend()
    ax.grid()
    
    # for agent in agents:
    #     # Write first three states to files.
    #     if idx_state == 0 or idx_state == 1 or idx_state == 2:
    #         # Data of first phase.
    #         trajectory_table = ""
    #         for j in range(t1, t_switch+1, step):
    #             trajectory_table += f"{j} {agent.cl_x[idx_state, j]}\n"

    #         # Write to file (no headers or footers, just the data)
    #         with open(f"./plotdata/{agent.id}_x{idx_state+1}_phase1.tex", "w") as f:
    #             f.write(trajectory_table)
                
    #         # Data of second phase.
    #         trajectory_table = ""
    #         for j in range(t_switch, t2, step):
    #             trajectory_table += f"{j} {agent.cl_x[idx_state, j]}\n"

    #         # Write to file (no headers or footers, just the data)
    #         with open(f"./plotdata/{agent.id}_x{idx_state+1}_phase2.tex", "w") as f:
    #             f.write(trajectory_table)

## --- Plot All 2 Inputs ---
for idx_input in range(num_inputs):
    if idx_input == 2:
        ax = axes[6, 0]
    else:
        ax = axes[5, idx_input]  # Get subplot position (last row)
    title_input = f'Closed-loop input $u_{idx_input+1}$ from t = {t1} to t = {t2_input}'

    for i, agent in enumerate(agents):
        tf = min(t2_input, agent.cl_u.shape[1] - 1)
        ax.plot(range(t1, tf+1), agent.cl_u[idx_input, t1:tf+1], 
                color=colours[i], label=f'A{agent.id}_u{idx_input+1}', markersize=0, linewidth=2, marker='o')

    if np.linalg.norm(ax.get_ylim()) < 1e-8:
        ax.set_ylim(-0.1, 0.1)
        
    if idx_input == 2:
        ax.set_ylim(0.0, 20.0)
    else:
        ax.set_ylim(-0.35, 0.35)
        
    ax.set_xlabel('$t$')
    ax.set_ylabel(f'$u_{idx_input+1}$')
    ax.set_title(title_input)
    # ax.legend()
    ax.grid()

# Adjust layout and show plot
plt.tight_layout()
plt.show()


In [ ]:
"""Collision"""
# Plot from t1 to t2.
t1 = 0 
t2 = max_sim_time + 1

t2 = min(t2, agents[0].cl_x.shape[1] - 1)  # Correct t2 if necessary.

fig_d, axes = plt.subplots(1, 2, figsize=(12, 6), num='distance', sharex=True)

ax_d = axes[0]  # Original plot
ax_zoom = axes[1]  # Zoomed-in plot

ax_d.set_xlabel('$t$') 
ax_d.set_ylabel(f'distance')
ax_d.set_title(f'Distance in position from t = {t1} to t = {t2}')
ax_d.grid(True)

considered_pairs = {}
j = 0
for i, agent in enumerate(agents):
    a1 = agent
    for a2 in agents:
        if (a2.id, a1.id) in considered_pairs or a1 == a2:
            continue
        else:
            considered_pairs[(a1.id, a2.id)] = True

        distances = []
        for t in range(t1, t2+1):
            distances.append(np.linalg.norm(a1.cl_x[0:3, t] - a2.cl_x[0:3, t])) 
            
        # Plot on both axes.
        for ax in [ax_d, ax_zoom]:
            ax.plot(range(t1, t2 + 1), distances, color=colours[j],
                    label=f'$\\Vert z_{{{a1.id[1:]}}} - z_{{{a2.id[1:]}}} \\Vert$',
                    markersize=0, linewidth=2, marker='o')
        j += 1

# Plot collision threshold on both axes.
for ax in [ax_d, ax_zoom]:
    ax.plot(range(t1, tf + 1),
            [data['MAS_parameters']['collision_distance']] * len(range(t1, tf + 1)),
            color='black', label='minimum distance', linewidth=2, linestyle='--')
    
# Labels, titles, and grid
ax_d.set_xlabel('$t$')
ax_d.set_ylabel(f"distance between agents' positions")
ax_d.set_title(f'Distance from t = {t1} to t = {t2}')
ax_d.grid(True)
ax_d.legend()

# Zoomed-in plot settings
y_min = data['MAS_parameters']['collision_distance'] - 0.05
y_max = data['MAS_parameters']['collision_distance'] + 0.05
ax_zoom.set_ylim(y_min, y_max)
ax_zoom.set_xlabel('$t$')
ax_zoom.set_title("Zoomed-in distance between agents' positions")
ax_zoom.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Set up figure and axes for side-by-side plots
fig, (ax_yT, ax_sp) = plt.subplots(1, 2, figsize=(12, 6), num='cooperation output')

# Extract the data:
t1 = 0
t2 = len(data['sim_data']['yT'][f'{agents[0].id}']) - 1

for i, agent in enumerate(agents):
    agent.yT_cl = [data['sim_data']['yT'][f'{agent.id}'][t] for t in range(0, t2+1)]
    
t_switch = data['cooperative_task']['switching_time']

# --- First Plot: Closed-loop evolution in state space ---
for i, agent in enumerate(agents):
    yT_cl = agent.yT_cl
    # First phase.
    ax_yT.plot([yT_cl[t][0] for t in range(t1, t_switch)],
               [yT_cl[t][1] for t in range(t1, t_switch)],
               color=colours[i], markersize=0, linewidth=1, marker='o',
               label=f'A{agent.id}_yT{2}')
    ax_yT.plot(yT_cl[0][0], yT_cl[0][1], color=colours[i], markersize=6, linewidth=1, marker='*')
    ax_yT.plot(yT_cl[t_switch][0], yT_cl[t_switch][1], color=colours[i], markersize=10, linewidth=1, marker='x')
    
    # Second phase.
    ax_yT.plot([yT_cl[t][0] for t in range(t_switch, t2)],
               [yT_cl[t][1] for t in range(t_switch, t2)],
               color=colours[i], markersize=0, linewidth=1, marker='o',
               label=f'A{agent.id}_yT{2}')
    ax_yT.plot(yT_cl[0][0], yT_cl[0][1], color=colours[i], markersize=6, linewidth=1, marker='*')
    ax_yT.plot(yT_cl[-1][0], yT_cl[-1][1], color=colours[i], markersize=10, linewidth=1, marker='x')

ax_yT.set_xlabel('$y_{T1}$')
ax_yT.set_ylabel('$y_{T2}$')
ax_yT.set_title('Closed-loop cooperation output on the 2D plane')
ax_yT.grid(True)
ax_yT.set_xlim(-2.5, 2.5)
ax_yT.set_ylim(-2.5, 2.5)
    

# Snapshots at switchting time and final time.
for i, agent in enumerate(agents):
    # Switching time.
    yT_cl = data['sim_data']['yT'][f'{agent.id}'][t_switch-1]
    ax_sp.plot(yT_cl[0, :], yT_cl[1, :], color=colours[i], markersize=3, linewidth=0.5, marker='o', label=f'A{agent.id}_yT')
    
    # Final time.
    yT_cl = data['sim_data']['yT'][f'{agent.id}'][-1]
    ax_sp.plot(yT_cl[0, :], yT_cl[1, :], color=colours[i], markersize=3, linewidth=0.5, marker='o', label=f'A{agent.id}_yT')
    

ax_sp.set_xlabel('$y_{T1}$')
ax_sp.set_ylabel('$y_{T2}$')
ax_sp.set_title('Cooperation output at switch and final time')
ax_sp.set_xlim(-2.5, 2.5)
ax_sp.set_ylim(-2.5, 2.5)
ax_sp.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
"""2D position with open-loop predictions"""
# Plot from t1 to t2.
t1 = 0
t2 = agent.cl_x.shape[1]-1
step = 1

# Select a feasible start time (the end time is controlled automatically).
t1 = min(t1, max_sim_time+1)

# Draw the evolution in state space:
fig_cl, ax_cl = plt.subplots(figsize=(7, 7))

for t in range(0, tf, step):
    for i, agent in enumerate(agents):
        ax_cl.plot(data['sim_data']['x'][f'{agent.id}'][t][0,:], data['sim_data']['x'][f'{agent.id}'][t][1,:], color=colours[i], linestyle='--', linewidth=1.0)
        
for i, agent in enumerate(agents):
    yT_cl = data['sim_data']['yT'][f'{agent.id}'][-1]
    ax_cl.plot(yT_cl[0,:], yT_cl[1,:], color=colours[i], markersize=0.1, linewidth=3, marker='o', label=f'A{agent.id}_yT', alpha=0.2)

ax_cl.set_xlabel('$x_1$')    
ax_cl.set_ylabel('$x_2$')
ax_cl.set_title(f'Open-loop predictions from $t = {t1}$ to $t = {t2}$ with step {step}')
ax_cl.set_xlim([-2.5, 2.5])
ax_cl.set_ylim([-2.5, 2.5])
ax_cl.grid()

plt.show()

In [ ]:
"""Animation"""
dark_rgb_colours = [
    (0.00, 0.45, 0.70),
    (0.83, 0.37, 0.00),
    (0.00, 0.62, 0.45),
    (0.94, 0.89, 0.26),
    (0.80, 0.47, 0.65),
    (0.34, 0.71, 0.91),
    (0.90, 0.56, 0.00),
    (0.60, 0.60, 0.60),
    (0.70, 0.13, 0.13),
    (0.42, 0.24, 0.60),
    (0.07, 0.45, 0.20), 
    (0.53, 0.80, 0.93),
    (0.87, 0.80, 0.47),
]

if animate:
    # Define the figure and axis
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-2.5, 2.5)
    ax.grid()
    ax.set_xlabel('$z^{\\mathrm{x}}$')
    ax.set_ylabel('$z^{\\mathrm{y}}$')

    # Store the maximum time index
    t1 = 0
    t2 = agents[0].cl_x.shape[1] - 1
    step = 1
    history_length = 30  # Number of previous steps to fade out

    # Plot elements to be updated
    agent_plots = []  # Stores the scatter objects
    trail_collections = []  # Stores the faded trail collections

    # Initialize the plot elements
    for i, agent in enumerate(agents):
        # Current position marker
        agent_plot, = ax.plot([], [], color=dark_rgb_colours[i], marker='o', markersize=6, linestyle='None')
        agent_plots.append(agent_plot)

        # Create a LineCollection for the fading trail with the correct colour
        trail_collection = LineCollection([], linewidth=1.5, colors=[dark_rgb_colours[i]])
        trail_collections.append(trail_collection)
        ax.add_collection(trail_collection)

    # Function to update the animation frame
    def update(frame):
        for i, agent in enumerate(agents):
            # Get the current and past positions
            current_x = np.array([agent.cl_x[0, frame]])  # Convert to sequence (array)
            current_y = np.array([agent.cl_x[1, frame]])

            past_start = max(0, frame - history_length)
            trail_x = agent.cl_x[0, past_start:frame+1]
            trail_y = agent.cl_x[1, past_start:frame+1]

            # Update the agent's position (scatter)
            agent_plots[i].set_data(current_x, current_y)

            # Create faded segments only if there are enough points
            if len(trail_x) > 1:
                segments = [((trail_x[j], trail_y[j]), (trail_x[j+1], trail_y[j+1])) for j in range(len(trail_x)-1)]
                
                # Generate alpha values for fading effect
                alpha_values = np.linspace(0.1, 1, len(segments))  # Fading from transparent (0.1) to opaque (1)

                # Convert agent color to RGBA and apply fading alpha
                faded_colors = [(dark_rgb_colours[i][0], dark_rgb_colours[i][1], dark_rgb_colours[i][2], alpha) for alpha in alpha_values]

                # Update the LineCollection with new segments and colors
                trail_collections[i].set_segments(segments)
                trail_collections[i].set_color(faded_colors)  # Set individual segment colors
            else:
                trail_collections[i].set_segments([])  # Clear trail if no valid segments
                
        # if frame == data['cooperative_task']['switching_time'] + 10:
        #     ax.set_xlim(-10, 10)
        #     ax.set_ylim(-10, 10)

        return agent_plots + trail_collections


    # Create animation (disable blitting for compatibility)
    ani = animation.FuncAnimation(fig, update, frames=range(t1, t2, step), interval=100, blit=False)

    filename = "./data/" + "quadrotor.mp4"
    ani.save(filename, writer="ffmpeg", dpi=300)

In [ ]:
"""3D animation"""
if animate:
    # Define the figure and 3D axis
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim(-3.5, 3.5)
    ax.set_ylim(-3.5, 3.5)
    ax.set_zlim(-3.5, 3.5)  # Adjust based on your drones' altitude range
    ax.set_xlabel('$z^{\\mathrm{x}}$')
    ax.set_ylabel('$z^{\\mathrm{y}}$')
    ax.set_zlabel('$z^{\\mathrm{z}}$')
    ax.grid()
    ax.view_init(elev=40, azim=-60)

    # Store the maximum time index
    t1 = 0
    t2 = agents[0].cl_x.shape[1] - 1
    step = 1
    history_length = 30  # Number of previous steps to fade out

    # Plot elements to be updated
    agent_plots = []  # Stores scatter objects
    trail_collections = []  # Stores 3D fading trail collections

    # Initialize plot elements
    for i, agent in enumerate(agents):
        # Current position marker (3D scatter)
        agent_plot = ax.scatter([], [], [], color=dark_rgb_colours[i], s=30)
        agent_plots.append(agent_plot)

        # Fading trail
        # Dummy segment to initialize safely
        dummy_segment = np.array([[[0, 0, 0], [0, 0, 0]]])

        trail_collection = Line3DCollection(dummy_segment, linewidths=1.5)
        trail_collections.append(trail_collection)
        ax.add_collection3d(trail_collection)

        # Immediately clear it (optional – the first frame update will override it anyway)
        trail_collection.set_segments([])

    # Function to update animation frames
    def update(frame):
        for i, agent in enumerate(agents):
            current_x = agent.cl_x[0, frame]
            current_y = agent.cl_x[1, frame]
            current_z = agent.cl_x[2, frame]

            # Update agent marker
            agent_plots[i]._offsets3d = ([current_x], [current_y], [current_z])

            # Generate trail history
            past_start = max(0, frame - history_length)
            trail_x = agent.cl_x[0, past_start:frame+1]
            trail_y = agent.cl_x[1, past_start:frame+1]
            trail_z = agent.cl_x[2, past_start:frame+1]

            if len(trail_x) > 1:
                segments = [((trail_x[j], trail_y[j], trail_z[j]),
                            (trail_x[j+1], trail_y[j+1], trail_z[j+1]))
                            for j in range(len(trail_x)-1)]

                alpha_values = np.linspace(0.1, 1, len(segments))
                faded_colors = [(dark_rgb_colours[i][0], dark_rgb_colours[i][1], dark_rgb_colours[i][2], alpha)
                                for alpha in alpha_values]

                trail_collections[i].set_segments(segments)
                trail_collections[i].set_color(faded_colors)
            else:
                trail_collections[i].set_segments([])

        if frame == data['cooperative_task']['switching_time'] - 20:
            ax.set_xlim(-10.5, 10.5)
            ax.set_ylim(-10.5, 10.5)
            ax.set_zlim(-3.5, 3.5)

        return agent_plots + trail_collections

    # Create and save animation
    ani = animation.FuncAnimation(fig, update, frames=range(t1, t2, step), interval=100, blit=False)

    filename = "./data/" + "quadrotor_3D_1.mp4"
    ani.save(filename, writer="ffmpeg", dpi=300)


In [ ]:
"""3D animation"""
if animate:
    # Define the figure and 3D axis
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim(-3.5, 3.5)
    ax.set_ylim(-3.5, 3.5)
    ax.set_zlim(-3.5, 3.5)  # Adjust based on your drones' altitude range
    ax.set_xlabel('$z^{\\mathrm{x}}$')
    ax.set_ylabel('$z^{\\mathrm{y}}$')
    ax.set_zlabel('$z^{\\mathrm{z}}$')
    ax.grid()
    ax.view_init(elev=0, azim=-60)

    # Store the maximum time index
    t1 = 0
    t2 = agents[0].cl_x.shape[1] - 1
    step = 1
    history_length = 30  # Number of previous steps to fade out

    # Plot elements to be updated
    agent_plots = []  # Stores scatter objects
    trail_collections = []  # Stores 3D fading trail collections

    # Initialize plot elements
    for i, agent in enumerate(agents):
        # Current position marker (3D scatter)
        agent_plot = ax.scatter([], [], [], color=dark_rgb_colours[i], s=30)
        agent_plots.append(agent_plot)

        # Fading trail
        # Dummy segment to initialize safely
        dummy_segment = np.array([[[0, 0, 0], [0, 0, 0]]])

        trail_collection = Line3DCollection(dummy_segment, linewidths=1.5)
        trail_collections.append(trail_collection)
        ax.add_collection3d(trail_collection)

        # Immediately clear it (optional – the first frame update will override it anyway)
        trail_collection.set_segments([])

    # Function to update animation frames
    def update(frame):
        for i, agent in enumerate(agents):
            current_x = agent.cl_x[0, frame]
            current_y = agent.cl_x[1, frame]
            current_z = agent.cl_x[2, frame]

            # Update agent marker
            agent_plots[i]._offsets3d = ([current_x], [current_y], [current_z])

            # Generate trail history
            past_start = max(0, frame - history_length)
            trail_x = agent.cl_x[0, past_start:frame+1]
            trail_y = agent.cl_x[1, past_start:frame+1]
            trail_z = agent.cl_x[2, past_start:frame+1]

            if len(trail_x) > 1:
                segments = [((trail_x[j], trail_y[j], trail_z[j]),
                            (trail_x[j+1], trail_y[j+1], trail_z[j+1]))
                            for j in range(len(trail_x)-1)]

                alpha_values = np.linspace(0.1, 1, len(segments))
                faded_colors = [(dark_rgb_colours[i][0], dark_rgb_colours[i][1], dark_rgb_colours[i][2], alpha)
                                for alpha in alpha_values]

                trail_collections[i].set_segments(segments)
                trail_collections[i].set_color(faded_colors)
            else:
                trail_collections[i].set_segments([])

        if frame == data['cooperative_task']['switching_time'] - 20:
            ax.set_xlim(-10.5, 10.5)
            ax.set_ylim(-10.5, 10.5)
            ax.set_zlim(-3.5, 3.5)

        return agent_plots + trail_collections

    # Create and save animation
    ani = animation.FuncAnimation(fig, update, frames=range(t1, t2, step), interval=100, blit=False)

    filename = "./data/" + "quadrotor_3D_2.mp4"
    ani.save(filename, writer="ffmpeg", dpi=300)
